# Predict DC Shared Bike Utilization Using Neural Network 
## Part I: Data Preparation

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras import optimizers
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

import warnings
warnings.filterwarnings("ignore", category = UserWarning, module = "matplotlib")

C:\Users\Xin\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### Data Loading

In [2]:
# Fetch orginal data from a bunch of csv files in one location. We downloaded the bike usage data in 2016 and 2017 online. 
# The size of total data is ~ 1GB. (https://s3.amazonaws.com/capitalbikeshare-data/index.html)
# The data is organized on a quarterly base.

# Load one csv data file and get a sense of data type and content for bike usage information

data_dir = 'original_trip_data/2016Q1-capitalbikeshare-tripdata.csv'

df_sample = pd.read_csv(data_dir)
display(df_sample.head(6))

,Duration,Start date,End date,Start station number,Start station,End station number,End station,Bike number,Member type
0,166,2016-01-01 00:06:58,2016-01-01 00:09:44,31102,11th & Kenyon St NW,31105,14th & Harvard St NW,W01346,Member
1,448,2016-01-01 00:10:20,2016-01-01 00:17:48,32039,Old Georgetown Rd & Southwick St,32002,Bethesda Ave & Arlington Rd,W22202,Member
2,715,2016-01-01 00:13:52,2016-01-01 00:25:48,31222,New York Ave & 15th St NW,31214,17th & Corcoran St NW,W21427,Member
3,213,2016-01-01 00:15:29,2016-01-01 00:19:03,31506,1st & Rhode Island Ave NW,31509,New Jersey Ave & R St NW,W01294,Member
4,872,2016-01-01 00:16:16,2016-01-01 00:30:49,31041,Prince St & Union St,31048,King St Metro South,W22058,Member
5,859,2016-01-01 00:16:18,2016-01-01 00:30:37,31041,Prince St & Union St,31048,King St Metro South,W01122,Casual


In [3]:
# Load all the data files into one dataframe
file_dir = 'original_trip_data/'
count = 0

for file_name in os.listdir(file_dir):
    count += 1
    file_handler = open(file_dir + file_name, 'r')
    print(file_name)
    
    # Only colect two columns
    columns = ['Start date','Member type']
    df= pd.read_csv(file_handler, usecols=columns)
     
    if(count == 1): data = df
    else: data = pd.concat([data,df])
     
    file_handler.close()

2016Q1-capitalbikeshare-tripdata.csv
2016Q2-capitalbikeshare-tripdata.csv
2016Q3-capitalbikeshare-tripdata.csv
2016Q4-capitalbikeshare-tripdata.csv
2017Q1-capitalbikeshare-tripdata.csv
2017Q2-capitalbikeshare-tripdata.csv
2017Q3-capitalbikeshare-tripdata.csv
2017Q4-capitalbikeshare-tripdata.csv


In [4]:
# Check if the data is loaded correctly
display(data.head(6))

,Start date,Member type
0,2016-01-01 00:06:58,Member
1,2016-01-01 00:10:20,Member
2,2016-01-01 00:13:52,Member
3,2016-01-01 00:15:29,Member
4,2016-01-01 00:16:16,Member
5,2016-01-01 00:16:18,Casual


### Data Cleaning on Bike Dataset

In [5]:
# Seperate detailed date/time information from 'Start date' due to the complexity in the format

data['weekday'] = pd.DatetimeIndex(data['Start date']).weekday
data['quarter'] = pd.DatetimeIndex(data['Start date']).quarter
data['year'] = pd.DatetimeIndex(data['Start date']).year
data['month'] = pd.DatetimeIndex(data['Start date']).month
data['day'] = pd.DatetimeIndex(data['Start date']).day
data['hour'] = pd.DatetimeIndex(data['Start date']).hour

# Check the format after processing
display(data.head(6))

,Start date,Member type,weekday,quarter,year,month,day,hour
0,2016-01-01 00:06:58,Member,4,1,2016,1,1,0
1,2016-01-01 00:10:20,Member,4,1,2016,1,1,0
2,2016-01-01 00:13:52,Member,4,1,2016,1,1,0
3,2016-01-01 00:15:29,Member,4,1,2016,1,1,0
4,2016-01-01 00:16:16,Member,4,1,2016,1,1,0
5,2016-01-01 00:16:18,Casual,4,1,2016,1,1,0


In [6]:
# Seperate Member type into Member and Casual and aggregate data by Member type

dummies = pd.get_dummies(data['Member type'], drop_first=False)
data = pd.concat([data, dummies], axis=1)

data['casual'] = data.groupby(['year', 'month', 'day', 'hour'])['Casual'].transform('sum')
data['registered'] = data.groupby(['year', 'month', 'day', 'hour'])['Member'].transform('sum')

# Check the format after processing
display(data.head(6))

,Start date,Member type,weekday,quarter,year,month,day,hour,Casual,Member,casual,registered
0,2016-01-01 00:06:58,Member,4,1,2016,1,1,0,0,1,9.0,47.0
1,2016-01-01 00:10:20,Member,4,1,2016,1,1,0,0,1,9.0,47.0
2,2016-01-01 00:13:52,Member,4,1,2016,1,1,0,0,1,9.0,47.0
3,2016-01-01 00:15:29,Member,4,1,2016,1,1,0,0,1,9.0,47.0
4,2016-01-01 00:16:16,Member,4,1,2016,1,1,0,0,1,9.0,47.0
5,2016-01-01 00:16:18,Casual,4,1,2016,1,1,0,1,0,9.0,47.0


In [7]:
# Select the date(year-month-day) from 'Start data' 
data['dteday'] = data['Start date'].str[:10]

display(data.head(6))

,Start date,Member type,weekday,quarter,year,month,day,hour,Casual,Member,casual,registered,dteday
0,2016-01-01 00:06:58,Member,4,1,2016,1,1,0,0,1,9.0,47.0,2016-01-01
1,2016-01-01 00:10:20,Member,4,1,2016,1,1,0,0,1,9.0,47.0,2016-01-01
2,2016-01-01 00:13:52,Member,4,1,2016,1,1,0,0,1,9.0,47.0,2016-01-01
3,2016-01-01 00:15:29,Member,4,1,2016,1,1,0,0,1,9.0,47.0,2016-01-01
4,2016-01-01 00:16:16,Member,4,1,2016,1,1,0,0,1,9.0,47.0,2016-01-01
5,2016-01-01 00:16:18,Casual,4,1,2016,1,1,0,1,0,9.0,47.0,2016-01-01


In [24]:
# Drop out non-relevant columns and duplicates (data aggregation)

df = data.drop(['Start date','Member type','Casual','Member'], axis=1)
print('The original dataframe contains ' + str(len(df)) + ' records.')

df.drop_duplicates(inplace = True)
df.reset_index(drop = True,inplace = True)

# Add the sum of casual and registered.
df['cnt'] = df.casual + df.registered

print('The final dataframe contains ' + str(len(df)) + ' records.')
display(df.head(6))

The original dataframe contains 7091771 records.
The final dataframe contains 17368 records.


,weekday,quarter,year,month,day,hour,casual,registered,dteday,cnt
0,4,1,2016,1,1,0,9.0,47.0,2016-01-01,56.0
1,4,1,2016,1,1,1,12.0,93.0,2016-01-01,105.0
2,4,1,2016,1,1,2,15.0,59.0,2016-01-01,74.0
3,4,1,2016,1,1,3,5.0,27.0,2016-01-01,32.0
4,4,1,2016,1,1,4,5.0,8.0,2016-01-01,13.0
5,4,1,2016,1,1,5,2.0,3.0,2016-01-01,5.0


In [25]:
# Add a new column 'holiday' to label whether it is holiday or not for a given date

holiday_list = ['2016-01-01', '2016-01-18', '2016-02-15', '2016-04-15', '2016-05-30', '2016-07-04',
               '2016-09-05', '2016-10-10', '2016-11-11', '2016-11-24', '2016-12-26','2017-01-02', 
                '2017-01-16', '2017-01-20', '2017-02-20', '2017-04-17', '2017-05-29', '2017-07-04',
               '2017-09-04', '2017-10-09', '2017-11-10', '2017-11-23', '2017-12-25']

df['holiday'] = df.dteday.isin(holiday_list).astype(int)

display(df.head(6))

,weekday,quarter,year,month,day,hour,casual,registered,dteday,cnt,holiday
0,4,1,2016,1,1,0,9.0,47.0,2016-01-01,56.0,1
1,4,1,2016,1,1,1,12.0,93.0,2016-01-01,105.0,1
2,4,1,2016,1,1,2,15.0,59.0,2016-01-01,74.0,1
3,4,1,2016,1,1,3,5.0,27.0,2016-01-01,32.0,1
4,4,1,2016,1,1,4,5.0,8.0,2016-01-01,13.0,1
5,4,1,2016,1,1,5,2.0,3.0,2016-01-01,5.0,1


### Data Cleaning on Weather Dataset

In [26]:
# Weather dataset is downloaded online. It contains weather information for 2016 and 2017.
'''
Abbreviation in weather dataset:  USAF = AIR FORCE CATALOG STATION NUMBER; SPD = WIND SPEED; 
CLG = CLOUD CEILING--LOWEST OPAQUE LAYER; TEMP & DEWP = TEMPERATURE & DEW POINT IN FAHRENHEIT; 
SLP = SEA LEVEL PRESSURE IN MILLIBARS
'''

weather = pd.read_csv('DC_weather_2016_2017.txt', delim_whitespace=True,
                   usecols = ['USAF', 'YR--MODAHRMN', 'SPD', 'CLG', 'TEMP', 'DEWP', 'SLP'])

weather = weather[weather['USAF'] == 724050]

display(weather.head(6))

weather.dtypes

C:\Users\Xin\Anaconda3\envs\keras\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,USAF,YR--MODAHRMN,SPD,CLG,TEMP,DEWP,SLP
0,724050,201601010452,11,44,45,32,1021.5
1,724050,201601010459,***,***,****,****,******
2,724050,201601010552,8,46,44,32,1021.1
3,724050,201601010600,8,***,44,32,1021.1
4,724050,201601010652,7,230,42,32,1020.8
5,724050,201601010752,8,220,42,32,1021.1


USAF             int64
YR--MODAHRMN     int64
SPD             object
CLG             object
TEMP            object
DEWP            object
SLP             object
dtype: object

There are missing data points labeled as ***.

In [27]:
# Replace **** with NaN for missing data points.

weather = weather.replace(['***','****','******'],np.nan)

In [28]:
# Calculate percentages of missing data in weather dataset

print('Percentage of missing data for TEMP: ', len(weather[weather['TEMP'].isna()])*1.0/len(weather))
print('Percentage of missing data for SLP: ', len(weather[weather['SLP'].isna()])*1.0/len(weather))
print('Percentage of missing data for SPD: ', len(weather[weather['SPD'].isna()])*1.0/len(weather))
print('Percentage of missing data for CLG: ', len(weather[weather['CLG'].isna()])*1.0/len(weather))
print('Percentage of missing data for DEWP: ', len(weather[weather['DEWP'].isna()])*1.0/len(weather))

Percentage of missing data for TEMP:  0.02559218728355728
Percentage of missing data for SLP:  0.19400193932677656
Percentage of missing data for SPD:  0.025938495636514753
Percentage of missing data for CLG:  0.20172461559772822
Percentage of missing data for DEWP:  0.025626818118853027


In [29]:
# Fill the missing data with the data point in the last record.

weather.fillna(method='ffill', inplace=True)

In [30]:
# Check if all the missing data points are filled out.

print('Percentage of missing data for TEMP: ', len(weather[weather['TEMP'].isna()])*1.0/len(data))
print('Percentage of missing data for SLP: ', len(weather[weather['SLP'].isna()])*1.0/len(data))

Percentage of missing data for TEMP:  0.0
Percentage of missing data for SLP:  0.0


In [31]:
# Date/time format conversion

weather['time'] = pd.DatetimeIndex(weather['YR--MODAHRMN'].astype(str))

# convert GMT time to DC time
weather['time_adjust'] = weather['time'] - np.timedelta64(4, 'h')

# Separate date/time information 

weather['year'] = pd.DatetimeIndex(weather['time_adjust']).year
weather['month'] = pd.DatetimeIndex(weather['time_adjust']).month
weather['day'] = pd.DatetimeIndex(weather['time_adjust']).day
weather['hour'] = pd.DatetimeIndex(weather['time_adjust']).hour

display(weather.head(5))

,USAF,YR--MODAHRMN,SPD,CLG,TEMP,DEWP,SLP,time,time_adjust,year,month,day,hour
0,724050,201601010452,11,44,45,32,1021.5,2016-01-01 04:52:00,2016-01-01 00:52:00,2016,1,1,0
1,724050,201601010459,11,44,45,32,1021.5,2016-01-01 04:59:00,2016-01-01 00:59:00,2016,1,1,0
2,724050,201601010552,8,46,44,32,1021.1,2016-01-01 05:52:00,2016-01-01 01:52:00,2016,1,1,1
3,724050,201601010600,8,46,44,32,1021.1,2016-01-01 06:00:00,2016-01-01 02:00:00,2016,1,1,2
4,724050,201601010652,7,230,42,32,1020.8,2016-01-01 06:52:00,2016-01-01 02:52:00,2016,1,1,2


In [32]:
# Drop out unrelavent columns 

dfw = weather.drop(['USAF', 'YR--MODAHRMN','time','time_adjust'], axis = 1)
print('Dataframe size:', len(dfw))
display(dfw.head(5))

# Remove duplicates (data aggregation)
dfw.drop_duplicates(subset=['year','month','day','hour'],inplace = True)
dfw.reset_index(drop = True,inplace = True)
print('Adjusted dataframe size:', len(dfw))
display(dfw.head(5))

Dataframe size: 28876


,SPD,CLG,TEMP,DEWP,SLP,year,month,day,hour
0,11,44,45,32,1021.5,2016,1,1,0
1,11,44,45,32,1021.5,2016,1,1,0
2,8,46,44,32,1021.1,2016,1,1,1
3,8,46,44,32,1021.1,2016,1,1,2
4,7,230,42,32,1020.8,2016,1,1,2


Adjusted dataframe size: 17538


,SPD,CLG,TEMP,DEWP,SLP,year,month,day,hour
0,11,44,45,32,1021.5,2016,1,1,0
1,8,46,44,32,1021.1,2016,1,1,1
2,8,46,44,32,1021.1,2016,1,1,2
3,8,220,42,32,1021.1,2016,1,1,3
4,7,220,42,30,1020.7,2016,1,1,4


In [33]:
dfw.dtypes

SPD      object
CLG      object
TEMP     object
DEWP     object
SLP      object
year      int64
month     int64
day       int64
hour      int64
dtype: object

In [34]:
# Convert the data type of weather from string to int

dfw['TEMP'] = dfw['TEMP'].astype(str).astype(int)
dfw['DEWP'] = dfw['DEWP'].astype(str).astype(int)
dfw['SPD'] = dfw['SPD'].astype(str).astype(int)
dfw['CLG'] = dfw['CLG'].astype(str).astype(int)
dfw['SLP'] = dfw['SLP'].astype(str).astype(float)

dfw.dtypes

SPD        int32
CLG        int32
TEMP       int32
DEWP       int32
SLP      float64
year       int64
month      int64
day        int64
hour       int64
dtype: object

## Data Merge

In [35]:
# Combine two dataframes aligned with date/time

df_combine = pd.merge(df, dfw, how='left', on=['year','month','day','hour'])

df_combine.head()

,weekday,quarter,year,month,day,hour,casual,registered,dteday,cnt,holiday,SPD,CLG,TEMP,DEWP,SLP
0,4,1,2016,1,1,0,9.0,47.0,2016-01-01,56.0,1,11.0,44.0,45.0,32.0,1021.5
1,4,1,2016,1,1,1,12.0,93.0,2016-01-01,105.0,1,8.0,46.0,44.0,32.0,1021.1
2,4,1,2016,1,1,2,15.0,59.0,2016-01-01,74.0,1,8.0,46.0,44.0,32.0,1021.1
3,4,1,2016,1,1,3,5.0,27.0,2016-01-01,32.0,1,8.0,220.0,42.0,32.0,1021.1
4,4,1,2016,1,1,4,5.0,8.0,2016-01-01,13.0,1,7.0,220.0,42.0,30.0,1020.7


In [36]:
print('Number of records in the combined dataframe: ', len(df_combine))

Number of records in the combined dataframe:  17368


In [37]:
# Check data types in the combined dataframe

df_combine.dtypes

weekday         int64
quarter         int64
year            int64
month           int64
day             int64
hour            int64
casual        float64
registered    float64
dteday         object
cnt           float64
holiday         int32
SPD           float64
CLG           float64
TEMP          float64
DEWP          float64
SLP           float64
dtype: object

In [38]:
# Fill out the missing datapoints

df_combine.fillna(method='ffill', inplace=True)
print(df_combine.isnull().values.any())

False


In [39]:
# Save the combined dataframe into a csv file

df_combine.to_csv('bike_weather_data.csv', index = False)